In [1]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten,Dropout,LSTM,Input,Embedding
from keras.callbacks import EarlyStopping
from keras.losses import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation


from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
import keras.backend as K
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

import gensim

wv = gensim.models.word2vec.Word2Vec.load("updated_google_corpus")

TESTING_MODE = True

Using TensorFlow backend.


In [34]:
# load dataset
train_data = pd.read_csv("train.csv")
#test_data = pd.read_csv("test.csv")
#test_data = test_data.drop("index",axis=1)
#data = pd.concat([train_data,test_data],sort=False)


MAX_LENGTH = train_data["description"].apply(lambda x:len(x.split())).max()

In [35]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [36]:
words = list(wv.wv.vocab.keys())
word_to_index, index_to_word = dict(),dict()
for i,word in enumerate(words):
    word_to_index[word]=i
    index_to_word[i]=word
print("vocabulary size= ",len(words)," words")

vocabulary size=  3064993  words


In [37]:
def vectorise(words,wv_model,max_length):
    words = words.lower().split()
    vectors = [wv_model[word] for word in words]
    return append_zeros(vectors,max_length)
def append_zeros(words,max_length):
    for i in range(max_length-len(words)):
        words.append(np.zeros(300))
    return np.array(words)

def sentences_to_indices(X, word_to_index, max_len):
    
    m = X.shape[0]
    
    X_indices = np.zeros((m, max_len),dtype=int)
    
    for i in range(m):
        
        sentence_words = X[i].split()
        
        j = 0
        for w in sentence_words:
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
            j = j+1
                
    return X_indices

In [38]:
# if TESTING_MODE:
#     d = pd.get_dummies(train_data[["country","region_1","region_2","province","taster_name","variety"]])
#     d["points"]= train_data["points"]
#     d["id"] = train_data['id']
# else:
#     d = pd.get_dummies(data[["country","region_1","region_2","province","taster_name","variety"]])
#     d["points"]= data["points"]
#     d["id"] = data['id']

In [39]:
d= pd.DataFrame()
d["id"] = train_data['id']
if TESTING_MODE:
    #d["description_vectors"] = train_data["description"].apply(lambda words:vectorise(words,wv,MAX_LENGTH))
    d["description"] = train_data["description"].apply(lambda x:x.lower())
#     for i in range(300):
#         d["vec_dim_"+str(i)]=description_vecs.apply(lambda v:v[i])
else:
    #d["description_vectors"] = train_data["description"].apply(lambda words:np.array([wv[word.lower()] for word in words.split()]))
    d["description"] = data["description"].apply(lambda x:x.lower())
#     for i in range(300):
#         d["vec_dim_"+str(i)] = description_vecs.apply(lambda v:v[i])

In [40]:
if TESTING_MODE:
    X_train, X_test, y_train, y_test = train_test_split(d, train_data["price"], test_size=0.3, random_state=1) # 70% training and 30% test
else:
    X_train,y_train,X_test, y_test = d[:175000], data["price"][:175000], d[175000:], data["price"][175000:]
    
X_train = X_train.drop(["id"],axis=1)

In [41]:
# X_train = np.array([X_train.values[i][0] for i in range(len(X_train.values))])
# X_train.shape
ids = X_test["id"]
X_test = X_test.drop(["id"],axis=1)
X_train = sentences_to_indices(np.array(X_train.values[:,0]),word_to_index,MAX_LENGTH)
X_test = sentences_to_indices(np.array(X_test.values[:,0]),word_to_index,MAX_LENGTH)

In [42]:
X_train.T.shape

(135, 122500)

In [43]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["hello"].shape[0]
    emb_matrix = np.zeros((vocab_len,emb_dim))
                                            
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
        
    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer


In [44]:
# model = Sequential()
# model.add(pretrained_embedding_layer(wv,word_to_index,input_shape=(MAX_LENGTH,300)))
# model.add(LSTM(128,return_sequences=False))
# model.add(Dropout(.5))
# model.add(Dense(1))
# model.add(Activation("relu"))

def deepLSTM(input_shape, dropout_prob,n_h):
    input_data = Input(shape=input_shape)
    embedding_layer = pretrained_embedding_layer(wv,word_to_index)(input_data)
    X = LSTM(n_h ,return_sequences=False)(embedding_layer)
    X = Dropout(dropout_prob)(X)
    X = Dense(1)(X)
    X = Activation("relu")(X)
           
    model = Model(inputs=input_data, outputs=X)
    return model
model = deepLSTM((MAX_LENGTH,), .5, 128)

In [45]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 135)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 135, 300)          919498200 
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 919,717,977
Trainable params: 219,777
Non-trainable params: 919,498,200
_______________________________________

In [46]:
model.compile(optimizer='rmsprop',
              loss=mean_squared_error,
              metrics=['accuracy'])


In [47]:
# X_test = np.array([X_test.values[i][0] for i in range(len(X_test.values))])
# X_test.shape

In [48]:
# X_train.values

In [49]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

In [50]:
X_train.shape

(122500, 135)

In [51]:
model.fit(X_train, y_train, epochs=100, batch_size=32,validation_data=(X_test, y_test),shuffle=True,callbacks=callbacks)

Train on 122500 samples, validate on 52500 samples
Epoch 1/100
122500/122500 [==============================] - 386s 3ms/step - loss: 1318.9156 - accuracy: 0.0252 - val_loss: 1292.1524 - val_accuracy: 0.0400
Epoch 2/100
122500/122500 [==============================] - 411s 3ms/step - loss: 1182.7583 - accuracy: 0.0291 - val_loss: 1185.8269 - val_accuracy: 0.0256
Epoch 3/100
122500/122500 [==============================] - 447s 4ms/step - loss: 1127.3442 - accuracy: 0.0296 - val_loss: 1133.2685 - val_accuracy: 0.0335
Epoch 4/100
122500/122500 [==============================] - 436s 4ms/step - loss: 1085.6768 - accuracy: 0.0310 - val_loss: 1139.0253 - val_accuracy: 0.0369
Epoch 5/100
122500/122500 [==============================] - 463s 4ms/step - loss: 1036.8334 - accuracy: 0.0320 - val_loss: 1100.5682 - val_accuracy: 0.0301
Epoch 6/100
122500/122500 [==============================] - 421s 3ms/step - loss: 1000.2141 - accuracy: 0.0319 - val_loss: 1077.3563 - val_accuracy: 0.0385
Epoch 7

In [52]:
model.save("lstm_encoder.model")

In [53]:
y_pred = model.predict(X_test)

In [54]:
print("Accuracy:",metrics.accuracy_score(y_test, np.int64(y_pred)))


Accuracy: 0.03485714285714286


In [55]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE: 30.678875651250603


In [61]:
# model.layers[:3].predict
model2 = Model(input = model.layers[0].input, output = model.layers[2].output)

In [77]:
print(X_test[0].shape)
model2.predict(X_test[0].reshape(1,135))
from keras.models import load_model
model3 = load_model("lstm_encoder.model")

(135,)


In [81]:
model2.predict(X_test[0].reshape(1,135))

array([[-2.16738731e-01, -9.55399632e-01,  0.00000000e+00,
         1.99299660e-02, -2.46438924e-02,  1.03284456e-01,
         1.00000000e+00, -1.00000000e+00, -4.35456680e-03,
        -0.00000000e+00,  9.99991119e-01,  0.00000000e+00,
        -8.23505130e-03, -0.00000000e+00, -8.96549225e-03,
         0.00000000e+00,  1.44540556e-02, -9.04576898e-01,
         0.00000000e+00, -9.99999642e-01,  0.00000000e+00,
        -3.59902173e-01, -6.42023422e-03,  0.00000000e+00,
         0.00000000e+00, -6.96942559e-04, -0.00000000e+00,
         0.00000000e+00, -3.53964090e-01, -0.00000000e+00,
         6.49516296e-04, -6.43325318e-03, -1.24359988e-01,
         3.98159027e-05,  0.00000000e+00,  1.49011598e-07,
         0.00000000e+00, -7.61594772e-01,  0.00000000e+00,
        -3.33819372e-09, -3.72914542e-07, -0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00,  9.50954735e-01,
         7.19632983e-01, -1.67262151e-08, -1.00000000e+0

In [79]:
model4 = Model(input = model.layers[0].input, output = model.layers[2].output)

In [80]:
model4.predict(X_test[0].reshape(1,135))

array([[-2.16738731e-01, -9.55399632e-01,  0.00000000e+00,
         1.99299660e-02, -2.46438924e-02,  1.03284456e-01,
         1.00000000e+00, -1.00000000e+00, -4.35456680e-03,
        -0.00000000e+00,  9.99991119e-01,  0.00000000e+00,
        -8.23505130e-03, -0.00000000e+00, -8.96549225e-03,
         0.00000000e+00,  1.44540556e-02, -9.04576898e-01,
         0.00000000e+00, -9.99999642e-01,  0.00000000e+00,
        -3.59902173e-01, -6.42023422e-03,  0.00000000e+00,
         0.00000000e+00, -6.96942559e-04, -0.00000000e+00,
         0.00000000e+00, -3.53964090e-01, -0.00000000e+00,
         6.49516296e-04, -6.43325318e-03, -1.24359988e-01,
         3.98159027e-05,  0.00000000e+00,  1.49011598e-07,
         0.00000000e+00, -7.61594772e-01,  0.00000000e+00,
        -3.33819372e-09, -3.72914542e-07, -0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00,  9.50954735e-01,
         7.19632983e-01, -1.67262151e-08, -1.00000000e+0